# Load Data

In [34]:
import pandas as pd
import numpy as np
import pickle
import time

In [35]:
start_time = time.time()

In [36]:
filename = '../../002_Data/Release_4/data_combined.pickle'
infile = open(filename,'rb')
df = pickle.load(infile)
infile.close()

filename = '../../002_Data/Release_4/attribute_list.pickle'
infile = open(filename,'rb')
keep = pickle.load(infile)
infile.close()

keep

['PAST_DUE',
 'TOTAL_60_DAYS_AMT',
 'NUM_PREM_FOR_PER',
 'BREAK_ARRANGEMENT',
 'COVID_REMINDER',
 'MULTI_DWELL_SIZE',
 'SNAP_GEO',
 'NUM_PER_FOR_PREM',
 'APARTMENT',
 'HAS_COTENANT']

In [37]:
print(df.BLOCKGROUP_GEOID.nunique())
print(df.POSTAL.nunique())

279
17


In [25]:
# causing convergence issues
keep.remove('APARTMENT')
#keep = keep + ['SPA_PER_ID', 'ARREARSMONTH', 'CMIS_MATCH']
keep.remove('SNAP_GEO')
keep = keep + ['SPA_PER_ID', 'ARREARSMONTH', 'CMIS_MATCH', 'POSTAL']

In [26]:
df = df[keep]

# Convert Format

## Goal
duration, variable values, event (0 or 1)

In [27]:
df = df.join(
    (df.groupby('SPA_PER_ID').ARREARSMONTH.max() - df.groupby('SPA_PER_ID').ARREARSMONTH.min()).rename('DURATION'),
    how='left',
    on='SPA_PER_ID',
)

In [28]:
# For left censoring
'''
df = df.join(
    df.groupby('SPA_PER_ID').ARREARSMONTH.min().rename('start'),
    how='left',
    on='SPA_PER_ID',
)
'''

"\ndf = df.join(\n    df.groupby('SPA_PER_ID').ARREARSMONTH.min().rename('start'),\n    how='left',\n    on='SPA_PER_ID',\n)\n"

## Take last values: most info about homelessness (best case)

In [29]:
print(len(df))
df = df.groupby('SPA_PER_ID').last().reset_index().drop('ARREARSMONTH', axis=1)
print(len(df))

3346276
85494


In [30]:
df.head()

,SPA_PER_ID,PAST_DUE,TOTAL_60_DAYS_AMT,NUM_PREM_FOR_PER,BREAK_ARRANGEMENT,COVID_REMINDER,MULTI_DWELL_SIZE,NUM_PER_FOR_PREM,HAS_COTENANT,CMIS_MATCH,POSTAL,DURATION
0,1,0,0.0,1,0,0,0,2,True,False,99217,1
1,3,0,0.0,1,0,0,0,1,False,False,99212,60
2,4,0,0.0,1,0,0,0,2,True,False,99216,35
3,6,0,0.0,1,0,0,0,1,False,False,99205,60
4,12,0,0.0,1,0,0,0,1,True,False,99208,15


# Dummy Variables

In [31]:
df = pd.get_dummies(data=df, prefix='ZIP', columns=['POSTAL'])

# Save

In [32]:
filename = '../../002_Data/Release_4/cox_PH_ready_geo.pickle'
outfile = open(filename, 'wb')
pickle.dump(df, outfile)
outfile.close()

# Time

In [33]:
from custom_methods.calc_time import calc_time_from_sec

calc_time_from_sec(time.time()-start_time)

hours:minutes:seconds = 0:0:3.968327522277832
